In [1]:
import os

In [2]:
%pwd

'/Users/yashlucky/Code/MLProjectsYoutube/chickenDiseaseClassification-mlproject/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/yashlucky/Code/MLProjectsYoutube/chickenDiseaseClassification-mlproject'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [7]:
from cdclassifier.constants import *
from cdclassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath = PARAMS_FILE_PATH
        ):
        
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)

        create_directories([self.config.artefacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url= config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        ) 

        return data_ingestion_config
        

In [2]:
import opendatasets as od

In [4]:
dataset = "https://www.kaggle.com/datasets/allandclive/chicken-disease-1"
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/allandclive/chicken-disease-1


100%|██████████| 266M/266M [00:53<00:00, 5.21MB/s] 
